In [124]:
import networkx as nx
import numpy as np
import scipy.optimize as sciop

class SupplyChain():
    def __init__(self):
         self.network=nx.MultiDiGraph()
    # add a factory/customer
    def addNode(self,myLabel,inDemand,outSupply):
        self.network.add_node(myLabel,demand=inDemand,supply=outSupply)
    def deleteNode(self,myLabel):
        self.network.remove_node(myLabel)
    def addEdge(self,node1,node2,myLabel,load,cost):
        self.network.add_edge(node1,node2,key=myLabel,name=myLabel,maxLoad=load,unitCost=cost,usedLoad=0)
    def deleteEdge(self,node1,node2,myLabel=None):
        if (myLabel is None):
            self.network.remove_edges_from([(node1,node2)])
        else:
            try:
                self.network.remove_edge(node1,node2,key=myLabel)
            except:
                print("No Such Key")
    def solveLP(self):
        edgeMatrixEntry={}
        i=0
        numEdges=len(self.network.edges())
        numNodes=len(self.network.nodes())
        numInequalities=numEdges+2*numNodes
        A = np.zeros((numInequalities,numEdges))
        b = np.zeros(numInequalities)
        c = np.zeros(numEdges)
        x = np.zeros(numEdges)
        for edge in self.network.edges(data=True):
            # add the inequality usedLoad <= maxLoad
            A[i,i]=1
            b[i]=edge[2]['maxLoad']
            # function to minimize has term unitCost*usedLoad
            c[i]=edge[2]['unitCost']
            # currentValue of usedLoad is best guess for optimal solution
            x[i]=edge[2]['usedLoad']
            edgeMatrixEntry[edge[2]['name']]=i
            i=i+1
        for node in self.network.nodes(data=True):
            # add the inequality that sum of incoming edges >= inDemand
            in_edges=self.network.in_edges(nbunch=node[0],data=True)
            for edge in in_edges:
                j=edgeMatrixEntry[edge[2]['name']]
                A[i,j]=-1
            b[i]=-node[1]['demand']
            # add the inequality that sum of outgoing edges <= outSupply
            out_edges=self.network.out_edges(nbunch=node[0],data=True)
            for edge in out_edges:
                j=edgeMatrixEntry[edge[2]['name']]
                A[i+1,j]=1
            b[i+1]=node[1]['supply']
            i=i+2
        print(A)
        print(b)
        print(c)
        print(x)
        # solve for the loads on the edges
        solutionX=sciop.linprog(c,A_ub=A,b_ub=b)
        for edge in self.network.edges(data=True):
            j = edgeMatrixEntry[edge[2]['name']]
            edge[2]['usedLoad']=solutionX['x'][j]
        return solutionX['x']

In [128]:
myNetwork=SupplyChain()
# two nodes with two roads. A cheap low capacity one and an expensive high capacity
myNetwork.addNode("name1",0,20)
myNetwork.addNode("name2",20,20)
myNetwork.addEdge("name1","name2","edge12",10,5)
myNetwork.addEdge("name1","name2","edge12_2",100,50)
# solve for the allotments
solution=myNetwork.solveLP()
print("Should put loads as "+str(solution))
print(myNetwork.network.nodes(data=True))
print(myNetwork.network.edges(data=True))
# try to remove an edge that isn't there, nothing changes
myNetwork.deleteEdge("name1","name2","junk")
solution=myNetwork.solveLP()
print("Should put loads as "+str(solution))
print(myNetwork.network.nodes(data=True))
print(myNetwork.network.edges(data=True))
# remove the cheap low capacity road
myNetwork.deleteEdge("name1","name2","edge12")
solution=myNetwork.solveLP()
print("Should put loads as "+str(solution))
print(myNetwork.network.nodes(data=True))
print(myNetwork.network.edges(data=True))

[[ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [-1. -1.]
 [ 0.  0.]]
[  10.  100.    0.   20.  -20.   20.]
[  5.  50.]
[ 0.  0.]
Should put loads as [ 10.  10.]
[('name1', {'demand': 0, 'supply': 20}), ('name2', {'demand': 20, 'supply': 20})]
[('name1', 'name2', {'name': 'edge12', 'maxLoad': 10, 'unitCost': 5, 'usedLoad': 10.0}), ('name1', 'name2', {'name': 'edge12_2', 'maxLoad': 100, 'unitCost': 50, 'usedLoad': 10.0})]
No Such Key
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [-1. -1.]
 [ 0.  0.]]
[  10.  100.    0.   20.  -20.   20.]
[  5.  50.]
[ 10.  10.]
Should put loads as [ 10.  10.]
[('name1', {'demand': 0, 'supply': 20}), ('name2', {'demand': 20, 'supply': 20})]
[('name1', 'name2', {'name': 'edge12', 'maxLoad': 10, 'unitCost': 5, 'usedLoad': 10.0}), ('name1', 'name2', {'name': 'edge12_2', 'maxLoad': 100, 'unitCost': 50, 'usedLoad': 10.0})]
[[ 1.]
 [ 0.]
 [ 1.]
 [-1.]
 [ 0.]]
[ 100.    0.   20.  -20.   20.]
[ 50.]
[ 10.]
Should put loads as [ 20.]
[('name1', {'demand': 0, 'supply'